# Notes on Python
## Examples for reference, tips, Best Practices

Based on the Course: Core Python (Implementing Iterators, Iterables and Collections) at PluralSight

Author: Gonçalo Felício  
Date: 04/2022  
Provided by: ISIWAY

Something like a pocketbook to come to for quick references, examples, and tips of best practices, compiled with my own preferences  
Loosely divided by subject, and with some degree, by the respective modules

Implementing custom iterators and iterables is very useful when handling specific data structures  
Much of the theory on this module is put in practice with the exercise module `Build a Personal Budget Report with Python Collections and Iterables`  
Can check the exercise on https://github.com/GoncaloFelicio/python-collections-budget, in specific the modules developed in the budget folder

`next(iterator)` delegates to `iterator.__next__()` which can be customized  
Iterators must support `__next__()`  
`__next__()` should return the next item in the series or raise a *StopIteration* if it's exhausted  

`iter(iterator)` delegates to `iterator.__iter__()` which can be customized  
Iterators must support `__iter__()`  
`__iter__()` should return an iterator for the given series


Iterators must be both support `__iter__()` and `__next__()`

In [1]:
# example of iterator from Budget exercise
class BudgetList():
    def __init__(self, budget):
        self.budget = budget
        self.sum_expenses = 0
        self.expenses = []
        self.sum_overages = 0
        self.overages = []


    def append(self, item):
        if (self.sum_expenses + item) < self.budget:
            self.expenses.append(item)
            self.sum_expenses += item
        else:
            self.overages.append(item)
            self.sum_overages += item


    def __len__(self):
        return sum([len(self.expenses), len(self.overages)])


    def __iter__(self):
        self.iter_e = iter(self.expenses)
        self.iter_o = iter(self.overages)
        return self


    def __next__(self):
        try:
            return self.iter_e.__next__()
        except StopIteration as stop:
            return self.iter_o.__next__()


Objects with a `__getitem__()` that accept consecutive integers starting from 0 are also iterables  
Iterables with this method raise a *IndexError* when exhausted  

Iterables can be called with the extended syntax that accepts 2 arguments: a *callable* to iterate over, and a *sentinel* that when returned from the callble stops the iteration

### Collection Protocols
`container` checks for membership

`sized` for length determination

`iterable` and `iterator` for collection traversal

`sequence` for random-access by index

`set` for managing collections of distinct elements

### Refactoring
Refactoring are transformations of the code that preserves correctness of the code and increases maintainability, readability and performance

For details we can check the relevant module to see `bisect` module applied to improve performance from O(n) to log(n), as well as an example of a `Test-Driven Development` in action

## Error Handling

You should always specify an exception type in an except statement otherwise you will hide exceptions that you do not intend

In [5]:
from random import randrange

def main():
    number = randrange(100)
    
    while True:
        try:
            guess = int(input('? '))
        except ValueError:
            continue
        if guess == number:
            print('Guessed right!')
            break
if __name__ == '__main__':
    main()

?  123
?  12
?  54
?  87
?  53
?  12


KeyboardInterrupt: Interrupted by user

?  2


In the previous exception, not specifying the error type would make the function impossible to stop with the *KeyboardInterrupt*

Exceptions are organized in hierarchies  
It is useful to know the hierarchy as we can catch an exception by its base class:
> for example *LookupError* includes both *IndexErro* and *KeyError*

On the downside, this method of error handling won't let us know exactly what went wrong, as it is ambiguous to which Error was raised exactly  
For this reason, in general, we want to be as specific as possible when catching exceptions, except when it's not important to know the exact error, as with the *OSError*, for example

Python Execptions carry a payload with more detailed on the condition that caused the exception  
Most built-in exceptions accept a string argument in the constructor which is stored as the payload  
Exceptions can also store data on a tuple argument called *args*, in practice, this argument should only have a single string 

In [10]:
def median(iterable):
    items = sorted(iterable)
    if len(items) == 0:
        raise ValueError('median() arg is an empty series, this is the custom string')
    
    median_index = (len(items) - 1) // 2
    if len(items) % 2 != 0:
        return items[median_index]
    return (items[median_index] + items[median_index + 1]) / 2

def main():
    try:
        median([])
    except ValueError as e:
        print('Payload repr:', repr(e))
        print('Payload str:', str(e))

    
if __name__ == '__main__':
    main()

Payload repr: ValueError('median() arg is an empty series, this is the custom string')
Payload str: median() arg is an empty series, this is the custom string


We can inherit from `Exception` to define a custom exception  
We can improve the custom exception by having it carry domain-relevant information to help diagnose problems  
We can then use the custom exception in calling code

In [47]:
import math

def triangle_area(a, b, c):
    sides = sorted((a, b, c))
    if sides[2] > sides[0] + sides[1]:
        raise TriangleException('Illegal Triangle', sides)
    
    p = (a + b + c) / 2
    a = math.sqrt(p * (p-a) * (p-b) * (p-c))
    return a

In [45]:
class TriangleException(Exception):
    
    def __init__(self, text, sides):
        super().__init__(text)
        self._sides = tuple(sides)
    
    @property
    def sides(self):
        return self._sides
    
    def __str__(self):
        return "'{}' for sides {}".format(self.args[0], self._sides)
    
    def __repr__(self):
        return "TriangleError({!r}, {!r})".format(self.args[0], self._sides)

In [46]:
triangle_area(3,4,10)

TypeError: exceptions must derive from BaseException

Exception chaining can be implicit or explicit:

Explicit chaining is when we use the statement `raise ..... from ..`  
Explicit chaining stores the original exception on `__cause__`

Implicit chaining is when an exception is raised during the handling of another exception  
Implicit chaining stores the original exception on `__context__`

#### Tracebacks
Tracebacks are objects like any other and can be interacted with  
They are stored on `__traceback__` on exceptions  
The `traceback` module provides fucntions to work with tracebacks
Storing tracebacks for too long is not good as it requires a large chunck of memory

In [51]:
from traceback import print_tb

def main():
    try:
        median([])
    except ValueError as e:
#         print('Payload repr:', repr(e))
#         print('Payload str:', str(e))
        
        print(e.__traceback__)
        print_tb(e.__traceback__)

if __name__ == '__main__':
    main()

  File "C:\Users\Goncalo\AppData\Local\Temp\ipykernel_14104\1340511165.py", line 5, in main
    median([])
  File "C:\Users\Goncalo\AppData\Local\Temp\ipykernel_14104\88029039.py", line 4, in median
    raise ValueError('median() arg is an empty series, this is the custom string')


Handling the traceback with the `print_tb` method we see that it prints the argument that causes the Exception and the raise statemetn as well

#### Assertions
Assertions can be used to guarantee that a function is returning what we think it's returning (Postconditions)  
Assertions are meant to detect errors in implementation, not user errors  
Therefore we need to careful not to raise an Assertion when the user is at fault, a raise an Exception instead  
Add guards to check input arguments and assertions to check implementation

#### Context Managers
The main use of context managers is for properly managing resources  
Context managers have `__enter__` and `__exit__` methods  
A `with` statement calls the `__enter__` before entering the block and the expression must evaluate to a context manager object  
The return value of `__enter__` is bound to the `as` variable, if defined  
The `__exit__` method is called after the block is complete  
If the with-block exits with an Exception, the information of the exception is still passed to the `__exit__` method  
By default the `__exit__` propagates exceptions, but we can hide it by returning `True`

Try switching True to False in the return of `__exit__`

In [69]:
class LoggingCM:
    def __enter__(self):
        print('Logging stuff from __enter__')
        return 'You have entered the with-block!'
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is None:
            print('Logging stuff from __exit__\n'
                 'Exit succeeded')
        else:
            print('Logging stuff from __exit__\n'
                 'Exit succee... WAIT! Exception detected.\n'
                 'type={}, value={}, traceback={}'.format(exc_type, exc_val, exc_tb))
        return True

In [70]:
with LoggingCM():
    pass

Logging stuff from __enter__
Logging stuff from __exit__
Exit succeeded


In [71]:
with LoggingCM():
    raise ValueError()

Logging stuff from __enter__
Logging stuff from __exit__
Exit succee... WAIT! Exception detected.
type=<class 'ValueError'>, value=, traceback=<traceback object at 0x000002156B0DC588>


#### Nested Context Managers
We can use multiple context managers by nesting them in the `with` statement separated by comma  
Careful not to pass a list to the expression of the with statement, and pass context managers!

In [75]:
import contextlib

@contextlib.contextmanager
def cm1(name):
    print(f'I am the {name}, a context manager')
    yield
    print(f'Exiting {name}')

@contextlib.contextmanager
def cm2(name):
    print(f'I am the {name}, a context manager too!')
    yield
    print(f'Exiting {name}')

In [76]:
with cm1('outer cm'), cm2('inner cm'):
    print('Body')

I am the outer cm, a context manager
I am the inner cm, a context manager too!
Body
Exiting inner cm
Exiting outer cm
